In [92]:
import pandas as pd 

flights = pd.read_csv('../../../large_files/flights_1_1000000.csv')

In [93]:
#flights.drop(columns=['rownum'],inplace=True)
cols = ['fl_date','op_unique_carrier','origin_airport_id','dest_airport_id','crs_dep_time','dep_time','dep_delay','taxi_out','wheels_off','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']
flights = flights[cols]

In [94]:
flights.sort_values(by=['op_unique_carrier','origin_airport_id','fl_date'],inplace=True)

In [95]:
carrier_ids = flights.op_unique_carrier.unique().tolist()

In [96]:
flights['rolling_dep_delay_carr-airport'] = 0
for i in carrier_ids:
    indy = flights[flights['op_unique_carrier']==i].index
    airport_ids = flights.loc[indy,'origin_airport_id'].unique().tolist()
    for j in airport_ids:
        indy2 = flights[flights['origin_airport_id']==j].index
        flights.loc[indy2,'rolling_dep_delay_carr-airport'] = flights.loc[indy2,'dep_delay'].rolling(30,min_periods=1).mean()

In [103]:
flights['month'] = pd.to_datetime(flights['fl_date'],format='%Y-%m-%d').apply(lambda x: x.month)
flights['year'] = pd.to_datetime(flights['fl_date'],format='%Y-%m-%d').apply(lambda x: x.year)
flights['day'] = pd.to_datetime(flights['fl_date'],format='%Y-%m-%d').apply(lambda x: x.day)
flights['day_of_week'] = pd.to_datetime(flights['fl_date'],format='%Y-%m-%d').apply(lambda x: x.dayofweek)
flights

,fl_date,op_unique_carrier,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,carrier_delay,...,security_delay,late_aircraft_delay,rolling_dep_delay_carr-airport,month,year,day_of_week,average_dep_delay_airport-month,hour,hourly_traffic,day
684941,2018-02-05,9E,10135,10397,1755,1758.0,3.0,15.0,1813.0,NaN,...,NaN,NaN,3.000000,2,2018,0,25.473251,18.0,77,5
713328,2018-02-06,9E,10135,10397,1755,1741.0,-14.0,13.0,1754.0,NaN,...,NaN,NaN,-5.500000,2,2018,1,25.473251,18.0,77,6
723313,2018-02-07,9E,10135,10397,1755,1751.0,-4.0,13.0,1804.0,NaN,...,NaN,NaN,-5.000000,2,2018,2,25.473251,18.0,77,7
741903,2018-02-08,9E,10135,10397,1755,1749.0,-6.0,18.0,1807.0,NaN,...,NaN,NaN,-5.250000,2,2018,3,25.473251,18.0,77,8
770477,2018-02-09,9E,10135,10397,1755,1804.0,9.0,13.0,1817.0,NaN,...,NaN,NaN,-2.400000,2,2018,4,25.473251,18.0,77,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957291,2018-02-18,ZW,15919,13930,1115,1134.0,19.0,11.0,1145.0,NaN,...,NaN,NaN,21.833333,2,2018,6,10.469477,12.0,0,18
979324,2018-02-19,ZW,15919,13930,1115,1110.0,-5.0,16.0,1126.0,NaN,...,NaN,NaN,19.800000,2,2018,0,10.469477,11.0,0,19
979398,2018-02-19,ZW,15919,13930,627,620.0,-7.0,16.0,636.0,NaN,...,NaN,NaN,19.633333,2,2018,0,10.469477,6.0,0,19
986764,2018-02-20,ZW,15919,13930,1115,1104.0,-11.0,12.0,1116.0,NaN,...,NaN,NaN,18.566667,2,2018,1,10.469477,11.0,0,20


In [98]:
flights['average_dep_delay_airport-month'] = 0
airport_ids = flights.origin_airport_id.unique().tolist()
for i in airport_ids:
    indy = flights[flights['origin_airport_id']==i].index
    temp = flights.loc[indy]
    months = temp.month.unique().tolist()
    for month in months:
        indy2 = temp[temp['month']==month].index
        flights.loc[indy2,'average_dep_delay_airport-month'] = flights.loc[indy2,'dep_delay'].mean()

In [99]:
flights.drop(columns = ['first_dep_time'],inplace=True)

In [100]:
flights['hour'] = round(flights.dep_time%100/60+(flights.dep_time-flights.dep_time%100)/100)

In [109]:
flights['hourly_traffic'] = 0
county = 0
for i in airport_ids:
    county+=1
    print(county)
    indy = flights[flights['origin_airport_id']==i].index
    temp = flights.loc[indy]
    years = temp['year'].unique().tolist()
    for year in years:
        indy2 = temp[temp.year==year].index
        temp2 = temp.loc[indy2]
        months = temp2['month'].unique().tolist()
        for month in months:
            indy3 = temp2[temp2.month==month].index
            temp3 = temp2.loc[indy3]
            days = temp3['day'].unique().tolist()
            for day in days:
                indy4 = temp3[temp3.day==day].index
                temp4 = temp3.loc[indy4]
                hours = temp4['hour'].unique().tolist()
                for hour in hours:
                    indy5 = temp4[temp4.hour==hour].index
                    flights.loc[indy5,'hourly_traffic'] = len(temp4.loc[indy5])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [113]:
flights = flights[~flights.carrier_delay.isna()]
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183716 entries, 816552 to 916166
Data columns (total 22 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   fl_date                          183716 non-null  object 
 1   op_unique_carrier                183716 non-null  object 
 2   origin_airport_id                183716 non-null  int64  
 3   dest_airport_id                  183716 non-null  int64  
 4   crs_dep_time                     183716 non-null  int64  
 5   dep_time                         183716 non-null  float64
 6   dep_delay                        183656 non-null  float64
 7   taxi_out                         183573 non-null  float64
 8   wheels_off                       183573 non-null  float64
 9   carrier_delay                    183716 non-null  float64
 10  weather_delay                    183716 non-null  float64
 11  nas_delay                        183716 non-null  float64
 1

In [ ]:
col_p = ['departures_performed', 'passengers', 'unique_carrier', 'month', 'year', 'origin_city_name', 'dest_city_name']
passengers = passengers[col_p]
passengers = passengers[passengers.passengers>0]
passengers = passengers[passengers.year>2018]

In [ ]:
op_unique_carrier = flights.op_unique_carrier.unique().tolist()
passengers = passengers[passengers.unique_carrier.isin(op_unique_carrier)]
passengers['passengers'] /= passengers.departures_performed
flights = flights.merge(passengers, how='inner', left_on=['op_unique_carrier','origin_city_name','dest_city_name','month','year'],left_on=['unique_carrier','origin_city_name','dest_city_name','month','year'])